In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
#import data
planet <- read.csv("../prep/point/merge_ready/planet_mr.csv")
sampsens <- read.csv("../prep/point/merge_ready/sample_sensor_mr.csv")
topo_1 <- read.csv("../prep/point/merge_ready/topo_1_mr.csv")
clusters <- read.csv("../prep/point/merge_ready/clusters.csv")

In [3]:
#merge datasets
by <- join_by(patch, date)

joined <- sampsens %>%
    left_join(planet, by = by) %>% # join planet data
    left_join(topo_1, "patch") %>% # join topo data
    left_join(clusters, "patch") %>%
    mutate(date = as.Date(date),
           crop = as.factor(crop),
           patch = as.factor(patch)) %>%
    filter(complete.cases(sm_60, sm_90, nmin_90)) #remove rows na value in these cols

# check for NAs
if (FALSE) {
    for (col in colnames(joined)){
        print(col)
        print(table(is.na(joined[[col]])))
    }
}

In [4]:
#calculate indicies

joined_indicies <- joined %>%
    mutate(
        NDVI = (band_8 - band_6) / (band_8 + band_6),    # Normalized Difference Vegetation Index (NDVI)
        GNDVI = (band_8 - band_4) / (band_8 + band_4),    # Green Normalized Difference Vegetation Index (GNDVI)
        EVI = 2.5 * (band_8 - band_6) / (band_8 + 6 * band_6 - 7.5 * band_2 + 1),    # Enhanced Vegetation Index (EVI)
        NDRE = (band_8 - band_7) / (band_8 + band_7),    # Normalized Difference Red Edge (NDRE)
        MCARI = ((band_6 - band_7) - 0.2 * (band_7 - band_4)) * (band_6 / band_7) / (band_8 + band_6 + band_7),    # Modified Chlorophyll Absorption Ratio Index (MCARI)
        LCCI = (band_8 - band_6) * (band_7 - band_4) / (band_8 + band_6) * (band_7 + band_4)     # Leaf Chlorophyll Content Index (LCCI)
    ) %>%
    relocate(nmin_90, .after = last_col()) %>%
    select(-date) #no need for date

In [5]:
write.csv(joined_indicies, "data/dataset_1.csv", row.names = FALSE)